In [ ]:
import os
import time
import requests
import numpy as np
import pandas as pd
import yfinance as yf
from datetime import datetime
import logging

def get_stock_codes():
    print('Downloading stock data...')
    urls = {
        2: "https://isin.twse.com.tw/isin/C_public.jsp?strMode=2",
        4: "https://isin.twse.com.tw/isin/C_public.jsp?strMode=4"
    }

    stock_dict = {}
    for mode, url in urls.items():
        res = requests.get(url)
        if res.status_code == 200:
            df = pd.read_html(res.text)[0]
            df.columns = ['full_name', 'isin_code', 'listed_date', 'market_type', 'industry_type', 'cfic_code', 'remarks']
            
            stock_start = df[df['full_name'].str.contains('股票', na=False)].index[0] + 1
            stock_end = df[df['full_name'].str.contains('上市認購\(售\)權證', na=False)].index[0] if mode == 2 else df[df['full_name'].str.contains('特別股', na=False)].index[0]
            
            stock_df = df.iloc[stock_start:stock_end]

            # Extract stock code and clean it
            stock_df['code'] = stock_df['full_name'].str.extract(r'(\d{4})')  # Extract numeric stock codes

            # Create dictionary with stock code as key and market type as value
            for _, row in stock_df.dropna(subset=['code']).iterrows():
                stock_dict[row['code']] = row['market_type']

    return stock_dict

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

# Define the output directory containing broker branch data files
output_dir = "../data_sample/chip/"

# Function to determine the correct Yahoo Finance stock suffix
def get_stock_suffix(market_type):
    return ".TW" if market_type == "上市" else ".TWO"

# Define price data directory
price_data_dir = "../data_sample/price/"
os.makedirs(price_data_dir, exist_ok=True)  # Ensure directory exists

# Function to check if historical price data is already cached
def get_price_cache_path(stock_code, end_date, period):
    return os.path.join(price_data_dir, f"{stock_code}_{end_date}_{period}.csv")

# Function to fetch historical stock prices with retry mechanism and caching
def query_historical_price(stock_code, market_type, end_date, period=390, max_retries=5, retry_delay=300):
    suffix = get_stock_suffix(market_type)
    start_date = (datetime.strptime(end_date, "%Y-%m-%d") - pd.Timedelta(days=period)).strftime("%Y-%m-%d")

    # Define cache file path
    cache_path = get_price_cache_path(stock_code, end_date, period)

    # Check if cached data exists
    if os.path.exists(cache_path):
        logging.info(f"Loading cached price data for {stock_code}{suffix} from {cache_path}")
        try:
            data = pd.read_csv(cache_path, parse_dates=['Date'])
            return data
        except Exception as e:
            logging.warning(f"Error reading cached file {cache_path}: {e}, refetching data...")

    logging.info(f"Fetching historical data for {stock_code}{suffix} from {start_date} to {end_date}")

    attempt = 0
    while attempt < max_retries:
        try:
            data = yf.download(f"{stock_code}{suffix}", start=start_date, end=end_date)
            if data.empty:
                logging.warning(f"No data found for {stock_code}{suffix}")
                return None

            data = data.reset_index()
            data['Date'] = pd.to_datetime(data['Date'])

            # Save fetched data to cache
            data.to_csv(cache_path, index=False)
            logging.info(f"Saved price data to {cache_path}")

            return data

        except requests.exceptions.ConnectionError as e:
            logging.error(f"Connection error fetching data for {stock_code}{suffix}: {e}")
            attempt += 1
            if attempt < max_retries:
                logging.info(f"Retrying in {retry_delay // 60} minutes... (Attempt {attempt}/{max_retries})")
                time.sleep(retry_delay)
            else:
                logging.error(f"Failed to fetch data for {stock_code}{suffix} after {max_retries} attempts.")
                return None
        except Exception as e:
            logging.error(f"Unexpected error fetching data for {stock_code}{suffix}: {e}")
            return None

# Global storage for broker branch performance tracking
global_wins = {}
global_trades = {}
global_turnover = {}
global_realized_profit = {}
global_unrealized_profit = {}
global_daily_returns = {}  # Key = (broker, branch), Value = list of tuples (date, return)
global_equity_curve = {}  # Key = (broker, branch), Value = list of tuples (date, equity)

# PnL computation logic using NET instead of separate Buy and Sell
def analyze_broker_data(file_path, stock_code, market_type, end_date):
    broker_data = pd.read_csv(file_path)
    broker_data['Date'] = pd.to_datetime(broker_data['date'])

    # Fetch historical prices
    historical_data = query_historical_price(stock_code, market_type, end_date)
    if historical_data is None:
        logging.error(f"Skipping {stock_code} due to missing price data.")
        return None

    merged_data = pd.merge(broker_data, historical_data, on='Date', how='left')
    last_close_price = historical_data['Close'].iloc[-1]

    # Track metrics
    results = {}

    for (broker, branch), group in merged_data.groupby(['broker', 'branch']):
        position = None  # Track only 1 active position at a time
        realized_profit = 0
        wins = 0
        trades = 0
        total_traded_value = 0  # Directly used as turnover now
        equity_curve = []
        daily_returns = []

        group = group.sort_values(by='Date')

        for _, row in group.iterrows():
            close_price = row['Close']
            if pd.isna(close_price):
                continue

            net = row['net'] if not pd.isna(row['net']) else 0
            current_date = row['Date']

            # Adjust net for open positions (closing trades)
            if position:
                if (position['type'] == 'long' and net < 0) or (position['type'] == 'short' and net > 0):
                    # Close existing position partially or fully
                    trade_qty = min(abs(net), position['amount'])
                    cost_of_trade = (trade_qty / position['amount']) * position['cost']

                    # Compute realized profit for the closed trade
                    profit = (
                        trade_qty * close_price - cost_of_trade
                        if position['type'] == 'long'
                        else cost_of_trade - trade_qty * close_price
                    )

                    # Compute daily return based on cost
                    if cost_of_trade > 0:
                        daily_return = profit / cost_of_trade
                        daily_returns.append((current_date, daily_return))

                    realized_profit += profit
                    if profit > 0:
                        wins += 1
                    trades += 1
                    total_traded_value += trade_qty * close_price

                    # Reduce position size or close it completely
                    position['amount'] -= trade_qty
                    net += trade_qty if position['type'] == 'long' else -trade_qty  # Adjust net
                    if position['amount'] == 0:
                        position = None

            # If `net` is in the same direction as an open position, increase it
            if position:
                if position['type'] == 'long' and net > 0:  # Increase long position
                    position['cost'] += abs(net) * close_price  # Add total cost, not average
                    position['amount'] += abs(net)  # Increase position size
                    total_traded_value += abs(net) * close_price
                    net = 0  # No new position needed
                elif position['type'] == 'short' and net < 0:  # Increase short position
                    position['cost'] += abs(net) * close_price  # Add total cost, not average
                    position['amount'] += abs(net)  # Increase position size
                    total_traded_value += abs(net) * close_price
                    net = 0  # No new position needed

            # Open a new position if net remains after closing old one
            if net > 0:  # Buy (net > 0)
                position = {'type': 'long', 'amount': abs(net), 'cost': abs(net) * close_price}
                total_traded_value += abs(net) * close_price
            elif net < 0:  # Sell (net < 0)
                position = {'type': 'short', 'amount': abs(net), 'cost': abs(net) * close_price}
                total_traded_value += abs(net) * close_price

            # Record realized profit for equity curve
            equity_curve.append((current_date, realized_profit))

        # Calculate unrealized profit
        unrealized_profit = 0
        if position:
            unrealized_profit = (
                (position['amount'] * last_close_price - position['cost'])
                if position['type'] == 'long'
                else (position['cost'] - position['amount'] * last_close_price)
            )

        # Compute performance metrics
        win_rate = (wins / trades) * 100 if trades > 0 else 0
        turnover = total_traded_value  # Updated turnover logic
        margin = realized_profit / max(total_traded_value, 1)  # Avoid div by zero

        # Compute drawdown using the worst negative PnL
        negative_pnl = [pnl for date, pnl in equity_curve if pnl < 0]  # Only negative values
        drawdown = min(negative_pnl) if negative_pnl else 0  # Worst loss

        # Compute Sharpe Ratio using valid daily returns
        if len(daily_returns) > 1:
            daily_returns_series = pd.Series([dr[1] for dr in daily_returns])
            sharpe_ratio = daily_returns_series.mean() / daily_returns_series.std() * np.sqrt(252) if daily_returns_series.std() > 0 else 0
        else:
            sharpe_ratio = 0

        # Store results
        results[(broker, branch)] = {
            'Realized Profit': realized_profit,
            'Unrealized Profit': unrealized_profit,
            'Win Rate (%)': win_rate,
            'Turnover': turnover,  # Directly use total_traded_value
            'Margin': margin,
            'Drawdown': drawdown,  # Now using worst negative PnL
            'Sharpe Ratio': sharpe_ratio
        }

        # Store global variables for tracking
        global_wins[(broker, branch)] = global_wins.get((broker, branch), 0) + wins
        global_trades[(broker, branch)] = global_trades.get((broker, branch), 0) + trades
        global_turnover[(broker, branch)] = global_turnover.get((broker, branch), 0) + turnover
        global_realized_profit[(broker, branch)] = global_realized_profit.get((broker, branch), 0) + realized_profit
        global_unrealized_profit[(broker, branch)] = global_unrealized_profit.get((broker, branch), 0) + unrealized_profit

        if (broker, branch) not in global_daily_returns:
            global_daily_returns[(broker, branch)] = []
        global_daily_returns[(broker, branch)].extend(daily_returns)

        if (broker, branch) not in global_equity_curve:
            global_equity_curve[(broker, branch)] = []
        global_equity_curve[(broker, branch)].extend(equity_curve)

    return pd.DataFrame.from_dict(results, orient='index')

# Load stock codes and their market type
stock_dict = get_stock_codes()

# Get all available broker branch data files
data_files = [f for f in os.listdir(output_dir) if f.startswith("broker_branch_data_") and f.endswith(".csv")]

# Initialize final results storage
all_results = []

# Process each file
for file in data_files:
    stock_code = file.split("_")[-1].split(".csv")[0]
    market_type = stock_dict.get(stock_code, "上市")
    file_path = os.path.join(output_dir, file)
    end_date = '2025-01-24'

    logging.info(f"Processing {file} for stock code {stock_code} ({market_type})")

    result_df = analyze_broker_data(file_path, stock_code, market_type, end_date)
    if result_df is not None:
        result_df['Stock Code'] = stock_code
        all_results.append(result_df)

# Combine all results
final_results = pd.concat(all_results, ignore_index=False)
final_results

In [ ]:
# Compute summary metrics for each broker branch directly from global dictionaries
summary_data = []

for (broker, branch) in global_wins.keys():
    wins = global_wins.get((broker, branch), 0)
    trades = global_trades.get((broker, branch), 1)  # Avoid division by zero
    turnover = global_turnover.get((broker, branch), 0)
    realized_profit = global_realized_profit.get((broker, branch), 0)
    unrealized_profit = global_unrealized_profit.get((broker, branch), 0)

    # Compute correct win rate percentage
    win_rate = (wins / trades) * 100 if trades > 0 else 0

    # Compute turnover as the total traded value
    avg_turnover = turnover  # Already aggregated in global dictionary

    # Compute margin as realized profit / total turnover
    avg_margin = realized_profit / max(turnover, 1)  # Avoid division by zero

    # Compute drawdown using worst negative PnL from equity curve
    equity_curve = global_equity_curve.get((broker, branch), [])
    negative_pnl = [pnl for date, pnl in equity_curve if pnl < 0]  # Only negative values
    drawdown = min(negative_pnl) if negative_pnl else 0  # Worst loss

    # Compute Sharpe Ratio using valid daily returns
    daily_returns = [dr[1] for dr in global_daily_returns.get((broker, branch), [])]
    if len(daily_returns) > 1:
        daily_returns_series = pd.Series(daily_returns)
        sharpe_ratio = daily_returns_series.mean() / daily_returns_series.std() * np.sqrt(252) if daily_returns_series.std() > 0 else 0
    else:
        sharpe_ratio = 0

    # Append results
    summary_data.append({
        'Broker': broker,
        'Branch': branch,
        'Realized Profit': realized_profit,
        'Unrealized Profit': unrealized_profit,
        'Win Rate (%)': win_rate,
        'Turnover': avg_turnover,
        'Margin': avg_margin,
        'Drawdown (%)': drawdown,
        'Sharpe Ratio': sharpe_ratio
    })

# Convert to DataFrame
summary = pd.DataFrame(summary_data)

summary